# NB1.5 — Streaming vs Batch (WebSockets & Polling)

En esta práctica implementamos **Streaming** con WebSockets y **Batch frecuente** con HTTP Polling. Guardaremos todo en **JSON Lines** en `../../data/` y compararemos latencia y continuidad.

**Objetivos**
- Entender diferencias clave **Batch vs Streaming**.
- Ejecutar un cliente **WebSocket** para ingesta continua.
- Ejecutar **HTTP Polling** como micro-batch.
- Comparar resultados en pandas.


## 1) Dependencias
Instalamos librerías requeridas para los scripts en `src/streaming/`.

In [4]:
!pip install -q websockets requests pandas

## 2) Streaming (WebSocket)

**Salida:** `../../data/stream_ws_YYYY-MM-DD.jsonl`

In [1]:
# Limita duración/eventos para la clase
!WS_MAX_EVENTS=150 WS_MAX_SECONDS=120 python ../../src/streaming/stream_dual_ws.py

Falta 'websockets'. Instala con: pip install websockets


## 3) Polling (HTTP)

**Salida:** `../../data/poll_bitcoin_YYYY-MM-DD.jsonl`

In [2]:
!python ../../src/streaming/poll_coincap_http.py

[POLL] 1/20 -> {'ts': '2025-09-27T18:33:06.467880+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 'price_usd': 109397.11}
[POLL] 2/20 -> {'ts': '2025-09-27T18:33:11.775899+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 'price_usd': 109397.11}
[POLL] 3/20 -> {'ts': '2025-09-27T18:33:17.087457+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 'price_usd': 109397.11}
[POLL] 4/20 -> {'ts': '2025-09-27T18:33:22.432894+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 'price_usd': 109397.11}
[POLL] 5/20 -> {'ts': '2025-09-27T18:33:27.753670+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 'price_usd': 109397.12}
[POLL] 6/20 -> {'ts': '2025-09-27T18:33:33.081616+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 'price_usd': 109397.12}
[POLL] 7/20 -> {'ts': '2025-09-27T18:33:38.401889+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 'price_usd': 109397.12}
[POLL] 8/20 -> {'ts': '2025-09-27T18:33:43.729264+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 

## 4) Cargar y comparar en pandas
Leemos ambos `.jsonl` y observamos diferencias de granularidad/latencia (aprox.).

In [7]:
from pathlib import Path
import pandas as pd

data_dir = Path("../../data/raw")

# STREAM
stream_files = sorted(data_dir.glob("stream_ws_*.jsonl"))
if stream_files:
    stream_file = stream_files[-1]
    df_stream = pd.read_json(stream_file, lines=True)
    print("STREAM:", stream_file)
    display(df_stream.head())
else:
    print("No se encontró ningún archivo de streaming (stream_ws_*.jsonl)")

# POLL
poll_files = sorted(list(data_dir.glob("poll_binance_*.jsonl")) + list(data_dir.glob("poll_coincap_*.jsonl")))
if poll_files:
    poll_file = poll_files[-1]
    df_poll = pd.read_json(poll_file, lines=True)
    print("POLL:", poll_file)
    display(df_poll.head())
else:
    print("No se encontró ningún archivo de polling (poll_binance_*.jsonl o poll_coincap_*.jsonl)")


No se encontró ningún archivo de streaming (stream_ws_*.jsonl)
POLL: ../../data/raw/poll_binance_BTCUSDT_2025-09-27.jsonl
POLL: ../../data/raw/poll_binance_BTCUSDT_2025-09-27.jsonl


,ts,source,instrument,price_usd
0,2025-09-27T18:33:06.467880+00:00,binance,BTCUSDT,109397.11
1,2025-09-27T18:33:11.775899+00:00,binance,BTCUSDT,109397.11
2,2025-09-27T18:33:17.087457+00:00,binance,BTCUSDT,109397.11
3,2025-09-27T18:33:22.432894+00:00,binance,BTCUSDT,109397.11
4,2025-09-27T18:33:27.753670+00:00,binance,BTCUSDT,109397.12


## 5) Reflexión (respuestas cortas)
**¿Quién tiene menor latencia?**  

**¿Qué pasa si el stream se cae?** 

**¿Cuál genera más duplicados o huecos temporales?**


**¿Quién tiene menor latencia?**
El streaming (WebSocket) tiene menor latencia porque recibe datos en tiempo real, mientras que el polling depende del intervalo de consulta.

**¿Qué pasa si el stream se cae?**
Si el stream se cae, se pierde la conexión y dejan de recibirse datos hasta que se reconecte. Es importante implementar reconexión automática para evitar huecos.

**¿Cuál genera más duplicados o huecos temporales?**
El polling puede generar más huecos temporales (por el intervalo entre consultas) y duplicados si el endpoint devuelve datos repetidos. El streaming es más continuo, pero puede tener huecos si la conexión falla.

**¿Quién tiene menor latencia?**  
El streaming (WebSocket) tiene menor latencia porque recibe datos en tiempo real, mientras que el polling depende del intervalo de consulta.

**¿Qué pasa si el stream se cae?**  
Si el stream se cae, se pierde la conexión y dejan de recibirse datos hasta que se reconecte. Es importante implementar reconexión automática para evitar huecos.

**¿Cuál genera más duplicados o huecos temporales?**  
El polling puede generar más huecos temporales (por el intervalo entre consultas) y duplicados si el endpoint devuelve datos repetidos. El streaming es más continuo, pero puede tener huecos si la conexión falla.